# LSTM Stock Predictor Using Closing Prices

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin closing prices to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [131]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [132]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [133]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('./data/btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [134]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('./data/btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [135]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [136]:
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [137]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [138]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 10

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [139]:
print (f"X sample values:\n{X[:3]} \n")
print (f"y sample values:\n{y[:3]}")

X sample values:
[[9114.719727 8870.820313 9251.269531 8218.049805 6937.080078 7701.25
  7592.720215 8260.69043  8696.830078 8569.290039]
 [8870.820313 9251.269531 8218.049805 6937.080078 7701.25     7592.720215
  8260.69043  8696.830078 8569.290039 8084.609863]
 [9251.269531 8218.049805 6937.080078 7701.25     7592.720215 8260.69043
  8696.830078 8569.290039 8084.609863 8911.269531]] 

y sample values:
[[8084.609863]
 [8911.269531]
 [8544.69043 ]]


In [140]:
# Use 70% of the data for training and the remaineder for testing
# YOUR CODE HERE!
split = int(.7 * len(X))
X_train = X[:split - 1]
X_test = X[split:]
y_train = y[:split - 1]
y_test = y[split:]

In [141]:
from sklearn.preprocessing import MinMaxScaler
# Use the MinMaxScaler to scale data between 0 and 1.
# YOUR CODE HERE!
#create scaler instances 
x_train_scaler = MinMaxScaler()
x_test_scaler = MinMaxScaler()
y_train_scaler = MinMaxScaler()
y_test_scaler = MinMaxScaler()

# Fit the scaler for the Training Data
x_train_scaler.fit(X_train)
y_train_scaler.fit(y_train)

# Scale the training data
X_train = x_train_scaler.transform(X_train)
y_train = y_train_scaler.transform(y_train)

# Fit the scaler for the Testing Data
x_test_scaler.fit(X_test)
y_test_scaler.fit(y_test)

# Scale the y_test data
X_test = x_test_scaler.transform(X_test)
y_test = y_test_scaler.transform(y_test)

In [142]:
X_train

array([[0.7111066 , 0.68162134, 0.72761425, ..., 0.60786209, 0.66058747,
        0.64516902],
       [0.68162134, 0.72761425, 0.60270722, ..., 0.66058747, 0.64516902,
        0.58657552],
       [0.72761425, 0.60270722, 0.44784942, ..., 0.64516902, 0.58657552,
        0.68651129],
       ...,
       [0.01963028, 0.01726567, 0.05168335, ..., 0.0451359 , 0.0430626 ,
        0.04363925],
       [0.01726567, 0.05168335, 0.05074402, ..., 0.0430626 , 0.04363925,
        0.04651042],
       [0.05168335, 0.05074402, 0.05471891, ..., 0.04363925, 0.04651042,
        0.05299984]])

Finally, reshape the X_train and X_test values to fit the model's requirement of samples, time steps, and features. (example: X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1)))

In [143]:
# Reshape the features for the model
# YOUR CODE HERE!
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [144]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

t - number of time steps
n - length of input vector in each time step
m - length of output vector (number of classes)
i - number of training examples
4(nm+n2)

In [145]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

# YOUR CODE HERE!
model = Sequential()
number_units = 30 # shape in output model summary of starter notebook 
dropout_fraction = 0.2 #20% 
#Layer 1 
model.add(LSTM(
    units=number_units, return_sequences=True,
    input_shape=(X_train.shape[1], 1)))
model.add(Dropout(dropout_fraction))
#Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
#Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
#output layer
model.add(Dense(1))


In [146]:
# Compile the model
# YOUR CODE HERE!
model.compile(optimizer='adam', loss='mean_squared_error')

In [147]:
# Summarize the model
# YOUR CODE HERE!
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 10, 30)            3840      
_________________________________________________________________
dropout_18 (Dropout)         (None, 10, 30)            0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 10, 30)            7320      
_________________________________________________________________
dropout_19 (Dropout)         (None, 10, 30)            0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 30)                7320      
_________________________________________________________________
dropout_20 (Dropout)         (None, 30)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                

In [148]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
# YOUR CODE HERE!
model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1, verbose=1)

Epoch 1/10
371/371 [==============================] - 9s 17ms/step - loss: 0.0276
Epoch 2/10
371/371 [==============================] - 7s 18ms/step - loss: 0.0241
Epoch 3/10
371/371 [==============================] - 6s 16ms/step - loss: 0.0290
Epoch 4/10
371/371 [==============================] - 5s 13ms/step - loss: 0.0282
Epoch 5/10
371/371 [==============================] - 5s 14ms/step - loss: 0.0246
Epoch 6/10
371/371 [==============================] - 6s 15ms/step - loss: 0.0230
Epoch 7/10
371/371 [==============================] - 5s 14ms/step - loss: 0.0226
Epoch 8/10
371/371 [==============================] - 5s 15ms/step - loss: 0.0198
Epoch 9/10
371/371 [==============================] - 5s 14ms/step - loss: 0.0199
Epoch 10/10
371/371 [==============================] - 5s 13ms/step - loss: 0.0184


---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [155]:
# Evaluate the model
# YOUR CODE HERE!
model.evaluate(X_test, y_test,return_dict=True)

5/5 [==============================] - 0s 11ms/step - loss: 0.0487


{'loss': 0.048721764236688614}

In [150]:
# Make some predictions
# YOUR CODE HERE!
predicted = model.predict(X_test)

In [151]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [152]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
stocks.head()

,Real,Predicted
2019-02-20,3406.344400,3698.748535
2019-02-21,3451.217446,3718.032959
2019-02-22,3417.875750,3742.047119
2019-02-23,3459.757801,3765.091553
2019-02-24,3608.907860,3785.775146


In [153]:
# Plot the real vs predicted values as a line chart
# YOUR CODE HERE!
stocks.head(100).hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)